In [1]:
import csv
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import pandas as pd

from keras.models import Sequential, Model
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
import gensim
import sys
import string
import logging
import random
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
in_domain_train = open("../dataset/cola_public_1.1/cola_public/raw/in_domain_train.tsv")
in_domain_train_tokenized = open("../dataset/cola_public_1.1/cola_public/tokenized/in_domain_train.tsv")
in_domain_dev = open("../dataset/cola_public_1.1/cola_public/raw/in_domain_dev.tsv")
in_domain_test = open("../dataset/cola-in-domain-open-evaluation/cola_in_domain_test.tsv")

In [5]:
read_train = csv.reader(in_domain_train_tokenized, delimiter="\t")
train = []
for row in read_train:
    print(row)
    train.append(row)

['gj04', '1', '', "our friends wo n't buy this analysis , let alone the next one we propose ."]
['gj04', '1', '', "one more pseudo generalization and i 'm giving up ."]
['gj04', '1', '', "one more pseudo generalization or i 'm giving up ."]
['gj04', '1', '', 'the more we study verbs , the crazier they get .']
['gj04', '1', '', 'day by day the facts are getting murkier .']
['gj04', '1', '', "i 'll fix you a drink ."]
['gj04', '1', '', 'fred watered the plants flat .']
['gj04', '1', '', 'bill coughed his way out of the restaurant .']
['gj04', '1', '', "we 're dancing the night away ."]
['gj04', '1', '', 'herman hammered the metal flat .']
['gj04', '1', '', 'the critics laughed the play off the stage .']
['gj04', '1', '', 'the pond froze solid .']
['gj04', '1', '', 'bill rolled out of the room .']
['gj04', '1', '', 'the gardener watered the flowers flat .']
['gj04', '1', '', 'the gardener watered the flowers .']
['gj04', '1', '', 'bill broke the bathtub into pieces .']
['gj04', '1', '', '

['ks08', '1', '', 'the clue that john got was more helpful than the one that smith got .']
['ks08', '1', '', 'the student was hoping for a tool .']
['ks08', '1', '', 'the tool that jones got was more helpful than the one that smith got .']
['ks08', '1', '', 'much evidence is needed .']
['ks08', '1', '', 'much equipment is needed .']
['ks08', '1', '', 'much information is needed .']
['ks08', '1', '', 'much furniture is needed .']
['ks08', '1', '', 'much advice is needed .']
['ks08', '0', '*', 'much clue is needed .']
['ks08', '0', '*', 'much tool is needed .']
['ks08', '0', '*', 'much armchair is needed .']
['ks08', '0', '*', 'much bags is needed .']
['ks08', '0', '*', 'many evidence was provided .']
['ks08', '0', '*', 'many equipment is available .']
['ks08', '0', '*', 'the room contains many furniture .']
['ks08', '1', '', 'the paper provides many clues .']
['ks08', '1', '', 'the box contains many tools .']
['ks08', '1', '', 'john offers many suggestions .']
['ks08', '1', '', 'little 

['m_02', '1', '', 'tess is knocking at the door .']
['m_02', '1', '', 'tess knocks at the door .']
['m_02', '1', '', 'frank churchill is crossing the street .']
['m_02', '1', '', 'frank churchill crosses the street .']
['m_02', '1', '', 'real play valencia next sunday .']
['m_02', '1', '', 'i leave for paris next week .']
['m_02', '0', '*', 'the volcano erupts on tuesday .']
['m_02', '1', '', 'the minister has arrived .']
['m_02', '1', '', "i 've been at work for six hours ."]
['m_02', '1', '', 'have you ever visited doubtful sound ?']
['m_02', '1', '', 'there was an attack yesterday .']
['m_02', '1', '', 'emma and harriet were attacked by those bandits .']
['m_02', '1', '', 'those bandits attacked emma and harriet yesterday .']
['m_02', '1', '', 'the vase was smashed deliberately .']
['m_02', '1', '', 'the sheep got infected with scrapie .']
['m_02', '1', '', 'the fans were deliberately provoked by a rival group .']
['m_02', '1', '', 'the fans got deliberately provoked by a rival grou

In [6]:
in_domain_train.close()

In [7]:
train

[['gj04',
  '1',
  '',
  "our friends wo n't buy this analysis , let alone the next one we propose ."],
 ['gj04', '1', '', "one more pseudo generalization and i 'm giving up ."],
 ['gj04', '1', '', "one more pseudo generalization or i 'm giving up ."],
 ['gj04', '1', '', 'the more we study verbs , the crazier they get .'],
 ['gj04', '1', '', 'day by day the facts are getting murkier .'],
 ['gj04', '1', '', "i 'll fix you a drink ."],
 ['gj04', '1', '', 'fred watered the plants flat .'],
 ['gj04', '1', '', 'bill coughed his way out of the restaurant .'],
 ['gj04', '1', '', "we 're dancing the night away ."],
 ['gj04', '1', '', 'herman hammered the metal flat .'],
 ['gj04', '1', '', 'the critics laughed the play off the stage .'],
 ['gj04', '1', '', 'the pond froze solid .'],
 ['gj04', '1', '', 'bill rolled out of the room .'],
 ['gj04', '1', '', 'the gardener watered the flowers flat .'],
 ['gj04', '1', '', 'the gardener watered the flowers .'],
 ['gj04', '1', '', 'bill broke the bathtu

In [8]:
train_array = np.array(train)
train_df = pd.DataFrame(train_array)

In [9]:
train_df

,0,1,2,3
0,gj04,1,,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,,one more pseudo generalization and i 'm giving...
2,gj04,1,,one more pseudo generalization or i 'm giving ...
3,gj04,1,,"the more we study verbs , the crazier they get ."
4,gj04,1,,day by day the facts are getting murkier .
...,...,...,...,...
8546,ad03,0,*,poseidon appears to own a dragon
8547,ad03,0,*,digitize is my happiest memory
8548,ad03,1,,it is easy to slay the gorgon .
8549,ad03,1,,i had the strangest feeling that i knew you .


In [12]:
read_train_tkn = csv.reader(in_domain_train_tokenized, delimiter="\t")
list_data = list(read_train_tkn)
list_data

[['gj04',
  '1',
  '',
  "our friends wo n't buy this analysis , let alone the next one we propose ."],
 ['gj04', '1', '', "one more pseudo generalization and i 'm giving up ."],
 ['gj04', '1', '', "one more pseudo generalization or i 'm giving up ."],
 ['gj04', '1', '', 'the more we study verbs , the crazier they get .'],
 ['gj04', '1', '', 'day by day the facts are getting murkier .'],
 ['gj04', '1', '', "i 'll fix you a drink ."],
 ['gj04', '1', '', 'fred watered the plants flat .'],
 ['gj04', '1', '', 'bill coughed his way out of the restaurant .'],
 ['gj04', '1', '', "we 're dancing the night away ."],
 ['gj04', '1', '', 'herman hammered the metal flat .'],
 ['gj04', '1', '', 'the critics laughed the play off the stage .'],
 ['gj04', '1', '', 'the pond froze solid .'],
 ['gj04', '1', '', 'bill rolled out of the room .'],
 ['gj04', '1', '', 'the gardener watered the flowers flat .'],
 ['gj04', '1', '', 'the gardener watered the flowers .'],
 ['gj04', '1', '', 'bill broke the bathtu

In [13]:
list_sent = [row[3] for row in list_data]
list_sent

["our friends wo n't buy this analysis , let alone the next one we propose .",
 "one more pseudo generalization and i 'm giving up .",
 "one more pseudo generalization or i 'm giving up .",
 'the more we study verbs , the crazier they get .',
 'day by day the facts are getting murkier .',
 "i 'll fix you a drink .",
 'fred watered the plants flat .',
 'bill coughed his way out of the restaurant .',
 "we 're dancing the night away .",
 'herman hammered the metal flat .',
 'the critics laughed the play off the stage .',
 'the pond froze solid .',
 'bill rolled out of the room .',
 'the gardener watered the flowers flat .',
 'the gardener watered the flowers .',
 'bill broke the bathtub into pieces .',
 'bill broke the bathtub .',
 'they drank the pub dry .',
 'they drank the pub .',
 'the professor talked us into a stupor .',
 'the professor talked us .',
 'we yelled ourselves hoarse .',
 'we yelled ourselves .',
 'we yelled harry hoarse .',
 'harry coughed himself into a fit .',
 'harry

In [14]:
list_sent_tkn = []

for sent in list_sent:
    list_sent_tkn.append(word_tokenize(sent))

list_sent_tkn

[['our',
  'friends',
  'wo',
  "n't",
  'buy',
  'this',
  'analysis',
  ',',
  'let',
  'alone',
  'the',
  'next',
  'one',
  'we',
  'propose',
  '.'],
 ['one',
  'more',
  'pseudo',
  'generalization',
  'and',
  'i',
  "'m",
  'giving',
  'up',
  '.'],
 ['one',
  'more',
  'pseudo',
  'generalization',
  'or',
  'i',
  "'m",
  'giving',
  'up',
  '.'],
 ['the',
  'more',
  'we',
  'study',
  'verbs',
  ',',
  'the',
  'crazier',
  'they',
  'get',
  '.'],
 ['day', 'by', 'day', 'the', 'facts', 'are', 'getting', 'murkier', '.'],
 ['i', "'ll", 'fix', 'you', 'a', 'drink', '.'],
 ['fred', 'watered', 'the', 'plants', 'flat', '.'],
 ['bill', 'coughed', 'his', 'way', 'out', 'of', 'the', 'restaurant', '.'],
 ['we', "'re", 'dancing', 'the', 'night', 'away', '.'],
 ['herman', 'hammered', 'the', 'metal', 'flat', '.'],
 ['the', 'critics', 'laughed', 'the', 'play', 'off', 'the', 'stage', '.'],
 ['the', 'pond', 'froze', 'solid', '.'],
 ['bill', 'rolled', 'out', 'of', 'the', 'room', '.'],
 ['the

In [31]:
w2v_model = Word2Vec(list_sent_tkn)

words = list(w2v_model.wv.index_to_key)
print(words)

['.', 'the', 'to', 'a', 'i', 'that', 'is', 'john', ',', 'of', 'he', 'and', 'was', 'in', '?', 'you', 'it', 'mary', 'be', 'who', 'did', 'on', 'bill', 'for', 'will', "'s", 'with', 'have', "n't", 'more', 'which', 'his', 'we', 'this', 'they', 'him', 'at', 'she', 'has', 'not', 'by', 'book', 'her', 'had', 'me', 'do', 'man', 'from', 'what', 'there', 'would', 'are', 'as', 'than', 'up', 'been', 'my', 'any', 'all', 'put', 'left', 'every', 'himself', 'can', 'some', 'into', 'gave', 'leave', 'ball', 'saw', 'eat', 'go', 'but', 'an', 'if', 'about', 'too', 'student', 'people', 'should', 'out', 'know', 'one', 'said', 'were', 'how', 'girl', 'students', 'kim', 'read', 'think', 'tom', 'much', 'so', 'picture', 'no', 'like', 'does', 'believe', 'bought', 'boy', 'could', 'your', 'table', 'when', 'ate', 'believed', 'never', 'made', 'books', 'fred', 'sent', 'tomorrow', 'seems', 'desk', 'sally', 'many', 'problem', 'red', 'got', 'eaten', 'before', 'came', 'hit', 'likes', 'might', 'them', 'everyone', 'last', 'want'

In [32]:
w2v_model.wv.most_similar('guitar')

[("'s", 0.9928901195526123),
 ('seemed', 0.9928472638130188),
 ('person', 0.9927605986595154),
 ('gets', 0.9926942586898804),
 ('eager', 0.9926020503044128),
 ('and', 0.9925433993339539),
 ('old', 0.992516815662384),
 ('mother', 0.9924942255020142),
 ('ate', 0.9924885630607605),
 ('are', 0.9924398064613342)]

In [33]:
w2v_weights = w2v_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape
MAX_SEQUENCE_LENGTH = 200

In [34]:
# X = train_df[3]
X = np.array(train_df[3].tolist())
y = np.array(train_df[1].tolist())

In [35]:
t = Tokenizer()
t.fit_on_texts(X)

In [45]:
# vocab_size = len(t.word_index) + 1
vocab_size, embedding_size = w2v_weights.shape

vocab_size

1712

In [37]:
encoded_docs = t.texts_to_sequences(X)

In [38]:
Xenc = pad_sequences(encoded_docs, maxlen=10, padding='post')
Xenc

array([[ 164,   31,  692, ...,   80,   30, 3523],
       [  80,   27, 2606, ..., 1470,   52,    0],
       [  80,   27, 2606, ..., 1470,   52,    0],
       ...,
       [  14,    6,  183, ...,    0,    0,    0],
       [   4,   41,    1, ...,  367,   13,    0],
       [  46,   56,   18, ...,    0,    0,    0]])

In [39]:
train_size=0.8

leinst = LabelEncoder()
yinst = leinst.fit_transform(y)

# X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(Xenc, yinst, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape), print(y_train.shape)
print(X_test.shape), print(y_test.shape)
print()

# test_size = 0.5
# X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=42)


# print(X_train.shape), print(y_train.shape)
print(X_validation.shape), print(y_validation.shape)
print(X_test.shape), print(y_test.shape)

(6840, 10)
(6840,)
(1711, 10)
(1711,)

(856, 10)
(856,)
(855, 10)
(855,)


(None, None)

In [40]:
# X_train = X_train.reshape((X_train.shape[0], 1))
# X_test = X_test.reshape((X_test.shape[0], 1))
# X_validation = X_validation.reshape((X_validation.shape[0], 1))

print(X_train.shape)
print(X_test.shape)
print(X_validation.shape)

num_classes = len(leinst.classes_)
print(num_classes)
print(leinst.classes_)

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_validation = to_categorical(y_validation, num_classes=num_classes)

(6840, 10)
(855, 10)
(856, 10)
2
['0' '1']


In [41]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [42]:
X_train

array([[   7,  368,    3, ...,    0,    0,    0],
       [  14,    6,    8, ...,    0,    0,    0],
       [   5, 4801, 4802, ...,   42, 1682,    0],
       ...,
       [   9, 1573,   27, ...,    0,    0,    0],
       [ 111,  121,   25, ...,   76,   26,    0],
       [   5,    1,   85, ...,   19,  100,  114]])

In [43]:
model = Sequential()

# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[w2v_weights],
                    input_length=10,
                    mask_zero=True,
                    trainable=False))

model.add(Bidirectional(LSTM(100)))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

ValueError: Layer embedding_2 weight shape (5393, 100) is not compatible with provided weight shape (1712, 100).

In [44]:
vocab_size

5393

In [28]:
model1 = Sequential()
model1.add(Input(shape=(10,)))
model1.add(Embedding(vocab_size,100,weights=[w2v_weights],input_length=10))
model1.add(Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat'))
model1.add(TimeDistributed(Dense(100,activation='relu')))
model1.add(Flatten())
model1.add(Dense(100,activation='relu'))
model1.add(Dense(3,activation='softmax'))
# model1 = Model(input1,output)
model1.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [29]:
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 100)           171200    
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 200)          160800    
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 10, 100)          20100     
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               100100    
                                                                 
 dense_3 (Dense)             (None, 3)                

In [30]:
history = model1.fit(X_train, y_train, epochs=5, batch_size=64,
                    validation_data=(X_validation, y_validation), verbose=1)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "c:\python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\python39\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\python39\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\python39\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\python39\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\python39\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "c:\python39\lib\asyncio\base_events.py", line 1890, in _run_once